In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import root_mean_squared_error

import pickle

In [2]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df['duration'] = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)] #filtering

    categorical = ['PULocationID', 'DOLocationID'] # we can extract the day of week, hour of day, month, etc from the datetime columns too
    df[categorical] = df[categorical].astype(str)

    return df

In [3]:
df_train = read_dataframe('./data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2024-02.parquet')

In [4]:
len(df_train), len(df_val)

(54373, 51497)

In [5]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [6]:
categorical =  ['PU_DO'] #['PULocationID', 'DOLocationID'] # we can extract the day of week, hour of day, month, etc from the datetime columns too
numerical = ['trip_distance']

dv = DictVectorizer() # turns dictionary to a vector
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)


In [7]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values   

In [8]:
linear = LinearRegression()
linear.fit(X_train, y_train)

y_pred = linear.predict(X_val)

RMSE = root_mean_squared_error(y_val, y_pred)
RMSE

5.995053551075027

In [9]:
lasso = Lasso()
lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_val)

RMSE = root_mean_squared_error(y_val, y_pred)
RMSE

9.110340896856446

In [10]:
ridge = Ridge()
ridge.fit(X_train, y_train)

y_pred = ridge.predict(X_val)

RMSE = root_mean_squared_error(y_val, y_pred)
RMSE

5.97931259382263

In [13]:
#saving the model
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, linear), f_out)

with open('models/lasso.bin', 'wb') as f_out:
    pickle.dump((dv, lasso), f_out)

with open('models/ridge.bin', 'wb') as f_out:
    pickle.dump((dv, ridge), f_out)